## Instacart Market Basket Analysis 

### a. Load the Data:
- Load the CSV files into pandas DataFrames.
- Check for missing values and data types.

In [1]:
import numpy as np
import pandas as pd

df_aisles = pd.read_csv('aisles.csv')
df_departments = pd.read_csv('departments.csv')
df_order_products_prior = pd.read_csv('order_products__prior.csv')
df_order_products_train = pd.read_csv('order_products__train.csv')
df_orders = pd.read_csv('orders.csv')
df_products = pd.read_csv('products.csv')


### set the empty value in days_since_prior_order to 0 as it is the first order for each user

In [2]:
#print(df_orders.loc[df_orders['days_since_prior_order'].isnull()])
df_orders.loc[df_orders['days_since_prior_order'].isnull(),'days_since_prior_order'] = 0
#remove user_id = 0 as all its data are 0
#i = df_orders[(df_orders.user_id == 0)].index
#df_orders.drop(i, inplace = True)
df_orders.dtypes

order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

### merge order_products prior and training tables

In [3]:
temp = df_order_products_train.copy()
temp.insert(0, 'istraining', 1)
df_order_products = df_order_products_prior.copy()
df_order_products.insert(0, 'istraining', 0)
frames = [df_order_products, temp]
df_order_products = pd.concat(frames, ignore_index=True)

In [4]:
#convert datatypes
#aisle
df_aisles['aisle_id'] = df_aisles['aisle_id'].astype('int64')
df_aisles['aisle'] = df_aisles['aisle'].astype('str')
#df_departments
df_departments['department_id'] = df_departments['department_id'].astype('int64')
df_departments['department'] = df_departments['department'].astype('str')
#df_products
df_products['product_id'] = df_products['product_id'].astype('int64')
df_products['product_name'] = df_products['product_name'].astype('str')
df_products['aisle_id'] = df_products['aisle_id'].astype('int64')
df_products['department_id'] = df_products['department_id'].astype('int64')
#df_order_products
df_order_products['product_id'] = df_order_products['product_id'].astype('int64')
df_order_products['order_id'] = df_order_products['order_id'].astype('int64')
df_order_products['add_to_cart_order'] = df_order_products['add_to_cart_order'].astype('int64')
df_order_products['reordered'] = df_order_products['reordered'].astype('bool')
df_order_products['istraining'] = df_order_products['istraining'].astype('bool')
#df_orders
df_orders['order_id'] = df_orders['order_id'].astype('int64')
df_orders['user_id'] = df_orders['user_id'].astype('int64')
df_orders['eval_set'] = df_orders['eval_set'].astype('str')
df_orders['order_number'] = df_orders['order_number'].astype('int64')
df_orders['order_dow'] = df_orders['order_dow'].astype('int64')
df_orders['order_hour_of_day'] = df_orders['order_hour_of_day'].astype('int64')
df_orders['days_since_prior_order'] = df_orders['days_since_prior_order'].astype('int64')

In [5]:

df_array = [
    df_aisles,
    df_departments,
    df_order_products,
    df_orders,
    df_products
]


### 3. Business Questions and Analysis
a. Popular Products:
- Analyze sales distribution of top-selling products.
- Identify top 5 products commonly added to the cart first.
- Top 10 product pairs frequently purchased together.

In [7]:
#Analyze sales distribution of top-selling products
order_products = df_order_products.groupby('product_id').agg({
    'order_id':'count'
}).nlargest(10,columns = 'order_id')
top_selling_products = order_products.head().index

#Identify top 5 products commonly added to the cart first
df_order_products[df_order_products.add_to_cart_order == 1].groupby('product_id').agg({
    'order_id':'count'
}).nlargest(5,columns = 'order_id')

#Top 10 product pairs frequently purchased together


,order_id
product_id,
24852,115521
13176,82877
27845,32071
21137,28875
47209,24913


b. Reorder Behavior:
- Analyze the reordered column to understand repeat purchase behavior.
- Products reordered the most.
- Reorder behavior based on day of the week and days since prior order.
- How the number of items in the cart impacts the likelihood of reordering.

In [15]:
#Analyze the reordered column to understand repeat purchase behavior
df_orders_products_join = df_order_products.join(df_order, how="inner", on = "order_id")
#Calculate the Proportion of Reorders
reorder_rate = df_orders_products_join['reordered'].mean()
print(f"Reorder Rate: {reorder_rate:.2%}")
#Analyze Reorder Rates by Product
product_reorder_rate = df_orders_products_join.groupby('product_id')['reordered'].mean()
top_reordered_products = product_reorder_rate.sort_values(ascending=False).head(10)
least_reordered_products = product_reorder_rate.sort_values().head(10)

print("Top 10 Reordered Products:\n", top_reordered_products)
print("Least 10 Reordered Products:\n", least_reordered_products)

#Analyze Reorder Rates by User
user_reorder_rate = df_orders_products_join.groupby('user_id')['reordered'].mean()
high_reorder_users = user_reorder_rate.sort_values(ascending=False).head(10)
low_reorder_users = user_reorder_rate.sort_values().head(10)

print("Top 10 Users by Reorder Rate:\n", high_reorder_users)
print("Bottom 10 Users by Reorder Rate:\n", low_reorder_users)

#Examine Reorder Behavior Over Time
reorder_rate_over_time = df_orders_products_join.groupby('order_number')['reordered'].mean()
reorder_rate_over_time.plot(title='Reorder Rate Over Time')

#Identify Most Frequently Reordered Items
frequent_reorders = order_number[order_number['reordered'] == 1]['product_id'].value_counts().head(10)
print("Most Frequently Reordered Items:\n", frequent_reorders)

#Cohort Analysis
#order_number['first_order_month'] = order_number.groupby('user_id')['order_id'].transform('min')
#cohort_reorder_rate = order_number.groupby(['first_order_month', 'order_number'])['reordered'].mean().unstack(0)
#cohort_reorder_rate.plot(title='Cohort Reorder Rate Over Time')

#----------------------------------------------------#
#Products reordered the most
df_order_products[df_order_products.reordered == 1].groupby('product_id').agg({
    'reordered':'count'
}).nlargest(10,columns = 'reordered')

#Reorder behavior based on day of the week and days since prior order.
reordered_orders = df_order_products[df_order_products.reordered == 1].order_id.unique()
df_orders[df_orders.order_id.isin(reordered_orders)].groupby('order_dow').agg({
    'order_id':'count'
}).nlargest(10,columns = 'order_id')
df_orders[df_orders.order_id.isin(reordered_orders)].groupby('days_since_prior_order').agg({
    'order_id':'count'
}).nlargest(10,columns = 'order_id')

#How the number of items in the cart impacts the likelihood of reordering.
#get number of products in reordered orders
reorder_order_index = df_order_products[df_order_products['reordered'] == 1]['order_id'].index
df_order_products[df_order_products.order_id.isin(reorder_order_index)].groupby('order_id').agg({
    'product_id':'count'
}).describe()

,product_id
count,2.948968e+06
mean,6.766896e+00
std,5.775252e+00
min,1.000000e+00
25%,3.000000e+00
50%,5.000000e+00
75%,9.000000e+00
max,1.300000e+02


c. Customer Segmentation:
- Segment customers based on the total amount they’ve spent on orders.
- Identify customers who haven’t placed an order in the last 30 days.
- Percentage of customers who have churned in the past quarter.

In [ ]:
from datetime import timedelta
#Months with higher order volumes
#assume start date 01012018 and all useres first order 
df_orders['date_of_order'] = pd.to_datetime('01/07/2018')
df_month = df_orders.groupby('user_id')
count1 = 0
#prev_date = pd.to_datetime('01/01/2018')
#current_user = 1
count2 =0

for user_id, group in df_orders.groupby('user_id'):
    prev_date = pd.to_datetime('01/07/2018')
    for index, row in group.iterrows():
        if row['order_number'] == 2:
            current_user = row['user_id']
            prev_date = pd.to_datetime('01/07/2018')+ timedelta(days=row['order_dow'])
            continue
        else:
            prev_date = prev_date + timedelta(days=row['days_since_prior_order'])
            df_orders.loc[((df_orders['order_id'] == row['order_id']) & (df_orders['user_id'] == row['user_id'])), 'date_of_order']\
            = pd.to_datetime(prev_date)

In [ ]:
df_order_products.head()

In [ ]:
'''
function takes number of products and classify customer based on following rules:
# Rare customer: Customers whose ordered products < 30 .
# Occasional customer: Customers whose ordered products < 100 .
# Frequent customer: Customers whose ordered products < 150.
# loyal customer: Customers whose ordered products > 150.
'''
'''
def customer_segment(order_prod):
    if order_prod < 30 :
        return 'RB'
    elif order_prod < 100:
        return 'OB'
    elif order_prod < 150:
        return 'FB'
    else:
        return 'LB'
#Segment customers based on the total amount they’ve spent on orders
#Segment customers based on the total number of products they ordered
customer_segment_order_products = df_order_products[['order_id','product_id']].merge(
    df_orders[['order_id','user_id']], 
    on = 'order_id',
    how = 'inner'
).groupby('user_id').agg({
    'product_id':'count'
})
product_count = list(customer_segment_order_products.product_id)
customer_segment_order_products = customer_segment_order_products.assign(
    segment = [customer_segment(x) for x in product_count]
)
customer_segment_order_products.head()
'''
#Identify customers who haven’t placed an order in the last 30 days.
latest_orderdate = df_orders['date_of_order'].max()
start_orderdate = latest_orderdate - timedelta(days=30)
df_orders[df_orders['date'].between(start_orderdate, latest_orderdate, inclusive='both')]

#Percentage of customers who have churned in the past quarter.

d. Department and Aisle Analysis:
- Best-selling department and aisle breakdown.
- What is the “produce” department? Break it down by aisle.
- Differences in purchasing behavior based on different departments or aisles

In [29]:
#Best-selling department and aisle breakdown.
Best_selling_products = list(df_order_products.groupby('product_id').agg({
    'order_id':'count'
}).nlargest(20,columns = 'order_id').index)

df_products[df_products.product_id.isin(Best_selling_products)].join(
    df_departments.set_index('department_id'), 
    on='department_id', 
    how = 'left'
).join(
    df_aisles.set_index('aisle_id'), 
    on='aisle_id', 
    how = 'left'
)




,product_id,product_name,aisle_id,department_id,department,aisle
5875,5876,Organic Lemon,24,4,produce,fresh fruits
8276,8277,Apple Honeycrisp Organic,24,4,produce,fresh fruits
13175,13176,Bag of Organic Bananas,24,4,produce,fresh fruits
16796,16797,Strawberries,24,4,produce,fresh fruits
21136,21137,Organic Strawberries,24,4,produce,fresh fruits
21902,21903,Organic Baby Spinach,123,4,produce,packaged vegetables fruits
22934,22935,Organic Yellow Onion,83,4,produce,fresh vegetables
24851,24852,Banana,24,4,produce,fresh fruits
24963,24964,Organic Garlic,83,4,produce,fresh vegetables
26208,26209,Limes,24,4,produce,fresh fruits


In [36]:
#What is the “produce” department? Break it down by aisle.
temp = df_products[df_products.department_id == 4].join(
    df_departments.set_index('department_id'), 
    on='department_id', 
    how = 'left'
).join(
    df_aisles.set_index('aisle_id'), 
    on='aisle_id', 
    how = 'left'
)
pd.unique(temp['aisle'])
#temp.head()

array(['packaged vegetables fruits', 'fresh vegetables', 'fresh fruits',
       'fresh herbs', 'packaged produce'], dtype=object)

In [13]:
#Differences in purchasing behavior based on different departments or aisles

In [12]:
 
purchasing_analysis = df_order_products.join(df_products, how = "left", on = "product_id")
                                  


ValueError: columns overlap but no suffix specified: Index(['product_id'], dtype='object')